# PRACTICA-09: **Datos geográficos con Basemap**

Un tipo común de visualización en la ciencia de datos es el de los *datos geográficos*. En esta práctica, mostraremos algunos ejemplos del tipo de visualización de mapas.

PASO-01: Instalar e importar el kit de herramientas de Basemap.

In [ ]:
!pip install basemap

In [ ]:
from mpl_toolkits import basemap
print(basemap.__version__)

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

In [ ]:
plt.figure(figsize=(8, 8))
m = Basemap(projection='ortho', resolution=None, lat_0=50, lon_0=-100)
m.bluemarble(scale=0.5);

Lo útil es que el globo que se muestra aquí no es una mera imagen; ¡Es un eje de Matplotlib completamente funcional que comprende coordenadas esféricas y que nos permite sobretrazar fácilmente datos en el mapa! Por ejemplo, podemos usar una proyección de mapa diferente, acercarnos a América del Norte y trazar la ubicación de Seattle. Usaremos una imagen etopo (que muestra características topográficas tanto en tierra como bajo el océano) como fondo del mapa:

In [ ]:
fig = plt.figure(figsize=(8, 8))
m = Basemap(projection='lcc', resolution=None,
            width=8E6, height=8E6, 
            lat_0=45, lon_0=-100,)
m.etopo(scale=0.5, alpha=0.5)

# Mapa (largo, lat) a (x,y) para trazar
x, y = m(-122.3, 47.6)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, ' Seattle', fontsize=12);

Esto da una breve idea del tipo de visualizaciones geográficas que son posibles. Ahora analizaremos las funciones de Basemap con más detalle y daremos algunos ejemplos de visualización de datos de mapas. 

# Proyecciones de mapas
Lo primero que se debe decidir al usar mapas es qué proyección usar. Probablemente estemos familiarizados con el hecho de que es imposible proyectar un mapa esférico, como el de la Tierra, sobre una superficie plana sin distorsionarlo de alguna manera o romper su continuidad. 

Dependiendo del uso previsto de la proyección del mapa, hay ciertas características del mapa (por ejemplo, dirección, área, distancia, forma u otras consideraciones) que es útil mantener.

Comenzaremos definiendo una rutina de conveniencia para dibujar nuestro mapa mundial junto con las líneas de **longitud** y **latitud**:

In [ ]:
from itertools import chain

def draw_map(m, scale=0.2):
    # dibujar una imagen en relieve sombreado
    m.shadedrelief(scale=scale)
    
    # latitudes y longitudes se devuelven como un diccionario
    lats = m.drawparallels(np.linspace(-90, 90, 13))
    lons = m.drawmeridians(np.linspace(-180, 180, 13))

    # claves contienen las instancias de plt.Line2D.
    lat_lines = chain(*(tup[1][0] for tup in lats.items()))
    lon_lines = chain(*(tup[1][0] for tup in lons.items()))
    all_lines = chain(lat_lines, lon_lines)
    
    # recorrer estas líneas y establecer el estilo deseado
    for line in all_lines:
        line.set(linestyle='-', alpha=0.3, color='w')

# Proyecciones cilíndricas
Las proyecciones cartográficas más simples son las *proyecciones cilíndricas*, en las que las líneas de **latitud** y **longitud** constantes se asignan a líneas horizontales y verticales, respectivamente. Este tipo de mapeo representa bastante bien las regiones ecuatoriales, pero produce distorsiones extremas cerca de los polos. 

El espaciado de las líneas de latitud varía entre diferentes proyecciones cilíndricas, lo que lleva a diferentes propiedades de conservación y diferentes distorsiones cerca de los polos. 

A continuación se muestra un ejemplo de la proyección cilíndrica equidistante, que elige una escala de latitud que preserva las distancias a lo largo de los meridianos. Otras proyecciones cilíndricas son las proyecciones de Mercator (projection='merc') y de áreas iguales cilíndricas (projection='cea').

In [ ]:
fig = plt.figure(figsize=(8, 6), edgecolor='w')
m = Basemap(projection='cyl', resolution=None,
            llcrnrlat=-90, urcrnrlat=90,
            llcrnrlon=-180, urcrnrlon=180, )
draw_map(m)

Los argumentos adicionales de Basemap para esta vista especifican la **latitud** (lat) y la **longitud** (lon) de la esquina inferior izquierda (llcrnr) y la esquina superior derecha (urcrnr) del mapa deseado, en unidades de grados.

# Proyecciones pseudocilíndricas
Las proyecciones pseudocilíndricas relajan el requisito de que los **meridianos** (líneas de longitud constante) permanezcan verticales; esto puede dar mejores propiedades cerca de los polos de la proyección. La *proyección de Mollweide* (projection='moll') es un ejemplo común de esto, en el que todos los meridianos son *arcos elípticos*. 

Está construido para preservar el área a lo largo del mapa: aunque hay distorsiones cerca de los polos, el área de pequeños parches refleja el área real. 

Otras proyecciones pseudocilíndricas son las proyecciones sinusoidal (projection='sinu') y Robinson (projection='robin').

In [ ]:
fig = plt.figure(figsize=(8, 6), edgecolor='w')
m = Basemap(projection='moll', resolution=None,
            lat_0=0, lon_0=0)
draw_map(m)

Los argumentos extra para Basemap aquí se refieren a la latitud central (lat_0) y la longitud (lon_0) para el mapa deseado.

# Proyecciones de perspectiva
Las proyecciones en perspectiva se construyen usando una elección particular de punto de perspectiva, similar a si fotografiaras la Tierra desde un punto particular en el espacio (un punto que, para algunas proyecciones, técnicamente se encuentra dentro de la Tierra). 

Un ejemplo común es la **proyección ortográfica** (projection='ortho'), que muestra un lado del globo visto desde una distancia muy larga. Como tal, solo puede mostrar la mitad del globo a la vez. 

Otras proyecciones basadas en perspectiva incluyen la proyección gnomónica (projection='gnom') y la proyección estereográfica (projection='stere'). Suelen ser los más útiles para mostrar pequeñas porciones del mapa.

Aquí hay un ejemplo de la proyección ortográfica:

In [ ]:
fig = plt.figure(figsize=(8, 8))
m = Basemap(projection='ortho', resolution=None,
            lat_0=50, lon_0=0)
draw_map(m);

# Proyecciones cónicas
Una proyección cónica proyecta el mapa en un solo cono, que luego se desenrolla. Esto puede generar muy buenas propiedades locales, pero las regiones alejadas del punto de enfoque del cono pueden distorsionarse mucho. 

Un ejemplo de esto es la proyección cónica conforme de Lambert (projection='lcc'). Proyecta el mapa en un cono dispuesto de tal manera que dos paralelos estándar (especificados en Basemap por lat_1 y lat_2) tienen distancias bien representadas, con una escala que disminuye entre ellos y aumenta fuera de ellos. 

Otras proyecciones cónicas útiles son la proyección cónica equidistante (projection='eqdc') y la proyección de áreas equivalentes de Albers (projection='aea'). 

Las proyecciones cónicas, como las proyecciones en perspectiva, tienden a ser buenas opciones para representar partes pequeñas o medianas del globo.

In [ ]:
fig = plt.figure(figsize=(8, 8))
m = Basemap(projection='lcc', resolution=None,
            lon_0=0, lat_0=50, lat_1=45, lat_2=55,
            width=1.6E7, height=1.2E7)
draw_map(m)

# Trazado de datos en mapas
Quizás la pieza más útil del kit de herramientas de Basemap es la capacidad de superponer una variedad de datos en un fondo de mapa. 

Para gráficos y texto simples, cualquier función plt funciona en el mapa; puede usar la instancia de Basemap para proyectar las coordenadas de latitud y longitud en coordenadas (x,y) para trazar con plt.


Algunos de estos métodos específicos del mapa son:

- **contour()/contourf()**: Dibujar líneas de contorno o contornos rellenos
- **imshow()**: Dibujar una imagen
- **pcolor()/pcolormesh()**: Dibuja un diagrama de pseudocolor para mallas irregulares/regulares
- **plot():** Dibujar líneas y/o marcadores.
- **scatter():** Dibujar puntos con marcadores.
- **carcaj():** Dibujar vectores.
- **drawgreatcircle():** Dibuja un gran círculo.

**Ejemplo:** Ciudades de California

Empezamos con la carga de los datos, como lo hemos hecho en actividades anteriores:

In [ ]:
!pip install basemap-data-hires

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

In [ ]:
import pandas as pd
cities = pd.read_csv('california_cities.csv')

# Extraer los datos que nos interesan
lat = cities['latd'].values
lon = cities['longd'].values
population = cities['population_total'].values
area = cities['area_total_km2'].values

A continuación, configuramos la proyección del mapa, dispersamos los datos y luego creamos una barra de colores y una leyenda:

In [ ]:
# 1. Dibujar el fondo del mapa
fig = plt.figure(figsize=(8, 8))
m = Basemap(projection='lcc', resolution='h', 
            lat_0=37.5, lon_0=-119,
            width=1E6, height=1.2E6)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')

# 2. Datos de ciudades dispersas, con población que refleja el color
# y tamaño del área.
m.scatter(lon, lat, latlon=True,
          c=np.log10(population), s=area,
          cmap='Reds', alpha=0.5)

# 3. Crear barra de colores y leyenda
plt.colorbar(label=r'$\log_{10}({\rm population})$')
plt.clim(3, 7)

# Hacer leyenda con puntos ficticios
for a in [100, 300, 500]:
    plt.scatter([], [], c='k', alpha=0.5, s=a,
                label=str(a) + ' km$^2$')
plt.legend(scatterpoints=1, frameon=False,
           labelspacing=1, loc='lower left');

Esto nos muestra aproximadamente dónde se han asentado poblaciones más grandes de personas en California: se agrupan cerca de la costa en las áreas de Los Ángeles y San Francisco, se extienden a lo largo de las carreteras en el valle central plano y evitan casi por completo las regiones montañosas a lo largo de las fronteras de el estado.